In [1]:
import time
M = 5
N = 5

In [2]:
class def_episode:
    def __init__(self, evnts, edge_set, lastevnt, pred, succ):
        self.freq = 0
        self.evnts = evnts.copy()
        self.edges = edge_set.copy()
        self.lastevnt = lastevnt
        self.pred = pred.copy()
        self.succ = succ.copy()
        # self.appclosed = True 

In [3]:
def CENOX_Span_Recursive(ParentBlock):
    global candidates, frequents, Freq_ep
    
    ChildBlock = list()
    parent_epi = ParentBlock[-1]
    last_event = parent_epi.lastevnt
    parent_len = len(parent_epi.evnts)
    
    Freq_ep.append(parent_epi)
    frequents += 1
    if len(parent_epi.evnts) >= N:
        return
    
    
    for helper_epi in ParentBlock:
        
        new_event = helper_epi.lastevnt
        Y0_events = parent_epi.evnts + [new_event]
        # Y0_edges = parent_epi.edges + helper_epi.edges
        Y0_edges = parent_epi.edges.copy()
        pred = helper_epi.pred
        succ = helper_epi.succ
        for ev_id in pred:
            Y0_edges.add((ev_id,parent_len))
        for ev_id in succ:
            Y0_edges.add((parent_len,ev_id))
        
        
        ValidEdges = ValidateEdges(parent_epi.pred,parent_epi.succ,helper_epi.pred,helper_epi.succ)

        if parent_epi.lastevnt != helper_epi.lastevnt:
            
            candidates += sum(ValidEdges)

            if ValidEdges[1]:
                Y1 = def_episode(Y0_events,Y0_edges|{(parent_len-1, parent_len)}, new_event, pred|{parent_len-1}, succ)
                ChildBlock.append(Y1)
                CENOX_Span_Recursive(ChildBlock,)
            
            if ValidEdges[0]:
                Y0 = def_episode(Y0_events, Y0_edges, new_event, pred, succ)
                ChildBlock.append(Y0)
                CENOX_Span_Recursive(ChildBlock)
            
            if ValidEdges[2]: 
                Y2 = def_episode(Y0_events,Y0_edges|{(parent_len, parent_len-1)}, new_event, pred, succ|{parent_len-1})
                ChildBlock.append(Y2)
                CENOX_Span_Recursive(ChildBlock)
        else:
            if ValidEdges[1]:
                candidates += 1
                Y0_edges.add((parent_len-1,parent_len))
                Y1 = def_episode(Y0_events,Y0_edges|{(parent_len-1, parent_len)}, new_event, pred|{parent_len-1}, succ)
                ChildBlock.append(Y1)
                CENOX_Span_Recursive(ChildBlock)

    return

In [4]:
def ValidateEdges(predp,succp,predh,succh):
    Validity = list([False,False,False])
    if (succp & predh):
        Validity[1] = True
    elif (succh & predp):
        Validity[2] = True
    else:
        Validity[0] = True
        if not (predp-predh | succh-succp):
            Validity[1] = True
        if not (predh-predp | succp-succh):
            Validity[2] = True
    return Validity

In [5]:
global candidates, frequents, Freq_ep

time_start = time.time()

alph = list()
F1_ep = list()
freq_alph = list()
for ev in range(M):
    alph.append(str(ev))
    F1_ep.append(def_episode([str(ev)],set(),str(ev),set(),set()))


candidates = len(alph)
frequents = 0
Freq_ep = list()
Block = list()
for parent_epi in F1_ep:
    Block.append(parent_epi)
    E = parent_epi.lastevnt
    CENOX_Span_Recursive(Block)
        

    
time_end = time.time()
time_run = time_end-time_start
print("time: ", time_run)
print("Number of closed frequent episodes is ", len(Freq_ep))

# print('The following are the frequent episodes discovered')
# for ep in Freq_closed_ep:
#     print(ep.freq ,':', ep.evnts)#, ep.edges) 


time:  0.5003988742828369
Number of closed frequent episodes is  49601


In [6]:
print(candidates, frequents, len(Freq_ep))

49601 49601 49601


In [7]:
level_counter = [0]*N
for ep in Freq_ep:
    level_counter[len(ep.evnts)-1] += 1
print(level_counter)

[5, 35, 315, 3480, 45766]


In [8]:
# for ep in Freq_ep:
#     if len(ep.evnts) == 3:
#         print(ep.evnts, ep.edges)